<a href="https://colab.research.google.com/github/sparks-baird/AxForChemistry/blob/main/tutorials/multi_objective_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Generate Sobol and multi-objective SAASBO Bayesian candidates for wetlab experiments

The first step is to generate a list of Sobol candidates, to be synthesized in a wetlab
environment or calculated if a simulation, etc. Once these candidates have been added
as a new .csv file (default is "post-sobol.csv") following the same format as the
training data, the second step will call `form.bayes_opt()`. At this point, it will suggest a batch of next best
experiments based on Pareto front-aware multi-objective optimization (MOO) to run for
a single adaptive design iteration. Note that the CSV file that gets loaded should be
stripped of any extra columns, otherwise these will be treated as search parameters. For
example, if you give it a CSV with multiple objectives and run a single-objective
optimization, the additional objectives will be erroneously considered as part of the
parameter search space.

For more information, see https://ax.dev/tutorials/saasbo_nehvi.html

Additional batches can then be generated. The setup of this tutorial assumes that there is some time between
when the experiments are suggested and when they are completed, and that experiments are
carried out "offline" (meaning this is not a closed-loop optimization process).

## Installation

In [4]:
!pip install axforchemistry

     |████████████████████████████████| 950 kB 6.1 MB/s 
     |████████████████████████████████| 718 kB 57.7 MB/s 
     |████████████████████████████████| 750.6 MB 11 kB/s 
     |████████████████████████████████| 347 kB 59.5 MB/s 
     |████████████████████████████████| 310 kB 67.7 MB/s 
  Created wheel for gpytorch: filename=gpytorch-1.6.0-py2.py3-none-any.whl size=509889 sha256=6b90663439be9ee0682456d2c9121c00230f7ee004f8725e87122e4ea419d074
  Stored in directory: /root/.cache/pip/wheels/66/b5/89/34c06ad393a6feb72b4cdde46d0f1c667f3e2632960f9df109
Successfully built gpytorch
  Attempting uninstall: torch
    Found existing installation: torch 1.10.0+cu111
    Uninstalling torch-1.10.0+cu111:
      Successfully uninstalled torch-1.10.0+cu111
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.11.1+cu111 requires torch==1.10.0, but you have torch 1.11.

## Imports

In [1]:
from os import path
from pathlib import Path
import numpy as np
import pandas as pd
from axforchemistry.axforchemistry_ import FormulationOptimization
from ax.service.utils.instantiation import ObjectiveProperties
from axforchemistry.utils.plotting import cv_plot
from sklearn.preprocessing import normalize # for dummy data
from sklearn.datasets import make_regression # for dummy data


C:\Users\sterg\Miniconda3\envs\axforchemistry\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Setup

To perform multi-objective optimization (MOO), specify the names of the objectives based
on the columns in the CSV file(s) of interest and whether the objective should be
minimized or maximized. `threshold == None` means to infer a threshold that the model
uses to help focus the search to a more useful range for the objective values. This
threshold acts as a soft constraint, and is set as a scalar value. For example, by
specifying `threshold=200` for the `"Compressive Strength (MPa)"` objective, where
greater is better (`minimize=False`), candidates that are likely to perform worse than this threshold are
less likely to be suggested as next experiments. In other words, this is a place where
you can bake-in domain knowledge to help the model decide what is useful or not.

In [2]:
compressive_key = "Compressive Strength (MPa)"
flexural_key = "Flexural Strength (MPa)"
vickers_key = "Vickers Hardness"
shrinkage_key = "Shrinkage (%)"
moo_objectives = {
    compressive_key: ObjectiveProperties(minimize=False, threshold=None),
    flexural_key: ObjectiveProperties(minimize=False, threshold=None),
    vickers_key: ObjectiveProperties(minimize=False, threshold=None),
    shrinkage_key: ObjectiveProperties(minimize=True, threshold=None),
}

data_dir = "data"
train_fname = "train-moo-fake.csv"
post_sobol_fname = "post-sobol-moo-fake.csv"

figdir = path.join("figures", "moo")

# trim the data down to the first 10 datapoints so it runs very fast (dummy run)
trim = False
if trim:
    df = pd.read_csv(path.join(data_dir, post_sobol_fname))
    df = df.head(10)
    post_sobol_fname = "post-sobol-moo-fake-dummy.csv"
    df.to_csv(path.join(data_dir, post_sobol_fname), index=False)


### Generate dummy data

The following is generated from a linear model and then the inputs are reworked to conform to our compositional constraint, such that each component is positive and `component_0 + component_1 + ... + component_n == 1.0`. We reparameterize this to `component_0 + component_1 + ... + component_{n-1} <= 1.0` to remove a degenerate dimension of the search and thereby increase the search efficiency.

In [7]:
n_samples = 100
n_features = 10
n_targets = len(moo_objectives)
X, y, coef = make_regression(n_samples=n_samples,
                       n_features=n_features,
                       n_informative=n_features-2,
                       n_targets=n_targets,
                       random_state=1,
                       coef=True)
X[X < 0.0] = -1 * X[X < 0.0]
X = 0.2 * normalize(X, norm="l1", axis=1)
y = X @ coef # recompute the targets
y = y + np.random.normal(size=(n_samples, n_targets), scale=10.0) # add noise
feat_columns = [f"component_{i}" for i in range(n_features)] # component_0, component_1, etc.
X_df = pd.DataFrame(X, columns=feat_columns)
targ_columns = [compressive_key, flexural_key, vickers_key, shrinkage_key]
y_df = pd.DataFrame(y, columns=targ_columns)
df = pd.concat((X_df, y_df), axis=1)
df.drop(columns=feat_columns[-1], inplace=True)
train_path = path.join(data_dir, train_fname)
Path(data_dir).mkdir(exist_ok=True, parents=True)
df.to_csv(train_path, index=False)
df

,component_0,component_1,component_2,component_3,component_4,component_5,component_6,component_7,component_8,Compressive Strength (MPa),Flexural Strength (MPa),Vickers Hardness,Shrinkage (%)
0,0.002403,0.034239,0.004367,0.004468,0.011043,0.053800,0.035516,0.015384,0.026792,-0.842453,6.378010,16.113686,-0.182803
1,0.031704,0.015127,0.000709,0.017091,0.003308,0.011786,0.069879,0.005531,0.033135,7.924150,34.715943,-12.922357,14.210516
2,0.028576,0.006921,0.006749,0.023859,0.032643,0.019061,0.035840,0.015922,0.024884,-4.763083,4.973592,12.775977,-12.304001
3,0.035261,0.000048,0.011189,0.032966,0.007851,0.017034,0.012578,0.045711,0.021746,3.218917,13.445902,8.737994,12.464910
4,0.019397,0.018287,0.015303,0.013213,0.032407,0.035748,0.045505,0.013697,0.006148,8.734551,15.655272,8.198462,39.635678
...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,0.035853,0.041514,0.029545,0.032611,0.004861,0.006579,0.008914,0.011108,0.025297,19.967873,0.787274,-10.575234,37.528432
96,0.021292,0.010481,0.015105,0.004949,0.012140,0.045672,0.032233,0.006331,0.017173,7.677474,14.213193,12.790518,14.493199
97,0.012554,0.023068,0.001666,0.035635,0.011997,0.012511,0.025997,0.032655,0.035409,-10.300538,24.042803,0.128413,-10.940054
98,0.011046,0.007264,0.010200,0.010444,0.023268,0.011309,0.046783,0.051603,0.008341,-5.650777,4.093064,-2.467657,13.962996


## Optimization

The optimization takes place with the FormulationOptimization class, which refers to
optimization of a formulation of components (i.e. `component_1 + component_2 +
component_3 + ... + component_n`) such that the sum of the fractional contributions of
all the components is equal to one.

In [8]:
# TODO: allow passing DataFrames directly
form = FormulationOptimization(
    train_max_val=0.4,  # i.e. i.e. x_1 + x_2 + x_3 + ... + x_{n-1} <= train_max_val
    sobol_max_val=0.25,  # i.e. x_1 + x_2 + x_3 + ... + x_{n-1} <= sobol_max_val
    sobol_min_val=0.15,  # i.e. x_1 + x_2 + x_3 + ... + x_{n-1} >= sobol_min_val
    bayes_max_val=None,  # default to Sobol equivalent
    bayes_min_val=None,  # default to Sobol equivalent
    seed=12345,
    n_bayes_batch=5,
    n_sobol=10,  # None --> 2*num_parameters
    num_samples=64,  # set to 256+ for real run (lower if OOM), 16 for dummy run
    warmup_steps=128,  # set to 512+ for real run (lower if OOM), 32 for dummy run
    exp_name="moo-example",
    moo_objectives=moo_objectives,
    exp_dir=path.join("experiments", "moo"),
    save_dir=path.join("results", "moo"),
    data_dir=data_dir,
    train_fname=train_fname,
    post_sobol_fname=post_sobol_fname,  # same format as `train_fname` + train data
)


loaded unique_components (ensure no extras):  ['component_0', 'component_1', 'component_2', 'component_3', 'component_4', 'component_5', 'component_6', 'component_7', 'component_8', 'last_component']


## Sobol candidates

First, we generate the suggested (pseudo-random) Sobol experiments to provide an initial
scaffolding for the initial model fit.

In [9]:
print("generating Sobol candidates and saving to .csv")
sobol_df, ax_sobol = form.train_and_sobol()
model = form.pre_sobol_model
sobol_df


[INFO 04-06 22:02:17] ax.service.utils.instantiation: Due to non-specification, we will use the heuristic for selecting objective thresholds.
[INFO 04-06 22:02:17] ax.service.utils.instantiation: Inferred value type of ParameterType.FLOAT for parameter component_0. If that is not the expected value type, you can explicity specify 'value_type' ('int', 'float', 'bool' or 'str') in parameter dict.
[INFO 04-06 22:02:17] ax.service.utils.instantiation: Inferred value type of ParameterType.FLOAT for parameter component_1. If that is not the expected value type, you can explicity specify 'value_type' ('int', 'float', 'bool' or 'str') in parameter dict.
[INFO 04-06 22:02:17] ax.service.utils.instantiation: Inferred value type of ParameterType.FLOAT for parameter component_2. If that is not the expected value type, you can explicity specify 'value_type' ('int', 'float', 'bool' or 'str') in parameter dict.
[INFO 04-06 22:02:17] ax.service.utils.instantiation: Inferred value type of ParameterType

generating Sobol candidates and saving to .csv


[INFO 04-06 22:02:17] ax.core.experiment: Attached data has some metrics ({'Flexural Strength (MPa)', 'Shrinkage (%)', 'Vickers Hardness'}) that are not among the metrics on this experiment. Note that attaching data will not automatically add those metrics to the experiment. For these metrics to be automatically fetched by `experiment.fetch_data`, add them via `experiment.add_tracking_metric` or update the experiment's optimization config.
[INFO 04-06 22:02:17] ax.core.experiment: Attached data has some metrics ({'Flexural Strength (MPa)', 'Shrinkage (%)', 'Vickers Hardness'}) that are not among the metrics on this experiment. Note that attaching data will not automatically add those metrics to the experiment. For these metrics to be automatically fetched by `experiment.fetch_data`, add them via `experiment.add_tracking_metric` or update the experiment's optimization config.
[INFO 04-06 22:02:17] ax.core.experiment: Attached data has some metrics ({'Flexural Strength (MPa)', 'Shrinkage

ValueError: StandardizeY transform requires non-empty observation data.

### Plotting

Now, we take a look at the cross-validation (CV) results for the SAASBO model using
the existing training data that was supplied to the model.

In [ ]:
figdir2 = path.join(figdir, "pre-sobol")
cv_results, fig, tile_fig = cv_plot(
    model,
    figdir=figdir2,
    fname="moo-cv",
    matplotlibify_kwargs=dict(height_inches=7.0, width_inches=7.0),
)


## Bayesian candidates

### First iteration
After completing the Sobol experiments (e.g. via wetlab synthesis and characterization)
and recording the measured objectives along with all of the available training data
within the `post_sobol_fname` file (e.g. `post-sobol-moo-fake.csv`), run the following cell
to generate the first batch of SAASBO Bayesian optimization candidates. The process is
then repeated: perform the suggested (real-world) experiments and run the script again
to get another batch of suggested candidates. This is meant to be an offline, manual
process geared towards manual experimental wetlab synthesis and characterization, though
more automated options exist.

In [ ]:
print("generating Bayes candidates and saving to .csv")
bayes_df, ax_bayes = form.bayes_opt()
model = ax_bayes.generation_strategy.model
bayes_df


At this point, you will run the SAASBO suggested experiments. Note that you are free
to run all of them, downselect, or modify the values of individual experiments, but if
you add or remove any parameters, these need to be represented for all variables. In the
case of a formulation where you decide to include a new component (e.g. a chemical that
you haven't used before), this is easy; simply add a column with `0.0` everywhere except
where you used the new chemical.

### Plotting

We can take a look at the cross-validation (CV) results for the SAASBO model using
whatever fully recorded data was made available to the model (i.e. existing training
data and Sobol data).

In [ ]:
figdir2 = path.join(figdir, "post-sobol")
cv_results, fig, tile_fig = cv_plot(
    model,
    figdir=figdir,
    fname="moo-cv",
    matplotlibify_kwargs=dict(height_inches=7.0, width_inches=7.0),
)

### Second iteration
Once you have finished running the experiments and have
updated the `post_sobol_fname` file (e.g. `post-sobol-moo-fake.csv`), then you can run
the second iteration of SAASBO suggested experiments.

In [ ]:
# ensure that `post_sobol_fname` file was updated with the new information.
print("generating Bayes candidates and saving to .csv")
bayes_df, ax_bayes = form.bayes_opt()
model = ax_bayes.generation_strategy.model
bayes_df


### Plotting

We can take a look at the cross-validation (CV) results for the SAASBO model after
the first iteration using whatever fully recorded data was made available to the model
(i.e. existing training data, Sobol data, and the first Bayesian batch).

In [ ]:
figdir2 = path.join(figdir, "bayes-0")
cv_results, fig, tile_fig = cv_plot(
    model,
    figdir=figdir,
    fname="moo-cv",
    matplotlibify_kwargs=dict(height_inches=7.0, width_inches=7.0),
)